In [79]:
# load dependencies
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import datetime

In [80]:
# connect to CryptoCompare API
api_key = os.environ.get("CryptoCompare_API_KEY")

In [81]:
# retrieve a list of all coins
url = f"https://min-api.cryptocompare.com/data/all/coinlist"
response = requests.get(url).json()

# convert to dataframe
crypto_list_df = pd.DataFrame(response["Data"]).transpose()

In [82]:
# visualize the dataframe
crypto_list_df.head()

,Id,Url,ImageUrl,ContentCreatedOn,Name,Symbol,CoinName,FullName,Description,AssetTokenStatus,...,MaxSupply,MktCapPenalty,IsUsedInDefi,IsUsedInNft,PlatformType,AlgorithmType,Difficulty,BuiltOn,SmartContractAddress,DecimalPoints
42,4321,/coins/42/overview,/media/35650717/42.jpg,1427211129,42,42,42 Coin,42 Coin (42),Everything about 42 coin is 42 - apart from th...,N/A,...,42,0,0,0,blockchain,scrypt,1.27287,NaN,NaN,NaN
300,749869,/coins/300/overview,/media/27010595/300.png,1517935016,300,300,300 token,300 token (300),300 token is an ERC20 token. This Token was cr...,N/A,...,300,0,0,0,token,NaN,NaN,ETH,0xaec98a708810414878c3bcdf46aad31ded4a4557,18
365,33639,/coins/365/overview,/media/352070/365.png,1480032918,365,365,365Coin,365Coin (365),365Coin is a Proof of Work and Proof of Stake ...,N/A,...,-1,0,0,0,blockchain,NaN,NaN,NaN,NaN,NaN
404,21227,/coins/404/overview,/media/35650851/404-300x300.jpg,1466100361,404,404,404Coin,404Coin (404),404 is a PoW/PoS hybrid cryptocurrency that al...,N/A,...,-1,0,0,0,blockchain,NaN,NaN,NaN,NaN,NaN
433,926547,/coins/433/overview,/media/34836095/433.png,1541597321,433,433,433 Token,433 Token (433),433 Token is a decentralised soccer platform t...,Finished,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
# get full column list
crypto_list_df.columns

Index(['Id', 'Url', 'ImageUrl', 'ContentCreatedOn', 'Name', 'Symbol',
       'CoinName', 'FullName', 'Description', 'AssetTokenStatus', 'Algorithm',
       'ProofType', 'SortOrder', 'Sponsored', 'Taxonomy', 'Rating',
       'IsTrading', 'TotalCoinsMined', 'CirculatingSupply', 'BlockNumber',
       'NetHashesPerSecond', 'BlockReward', 'BlockTime', 'AssetLaunchDate',
       'AssetWhitepaperUrl', 'AssetWebsiteUrl', 'MaxSupply', 'MktCapPenalty',
       'IsUsedInDefi', 'IsUsedInNft', 'PlatformType', 'AlgorithmType',
       'Difficulty', 'BuiltOn', 'SmartContractAddress', 'DecimalPoints'],
      dtype='object')

In [86]:
crypto_clean = crypto_list_df.loc[:, ["Name", "Symbol", "CoinName", "TotalCoinsMined", "AssetLaunchDate", "MktCapPenalty"]]

crypto_clean.head()

,Name,Symbol,CoinName,TotalCoinsMined,AssetLaunchDate,MktCapPenalty
42,42,42,42 Coin,41.999952,2014-01-05,0
300,300,300,300 token,300,2017-07-01,0
365,365,365,365Coin,0,2017-11-17,0
404,404,404,404Coin,0,2017-10-31,0
433,433,433,433 Token,NaN,NaN,NaN


In [89]:
# investigate the number of coins available
len(crypto_clean)

8374

In [ ]:
# we need to decide on 25 to ffc

In [76]:
def RetrieveDailyPrice(syms, comp_syms, all_data=True, exchange=''):
    url = "https://min-api.cryptocompare.com/data/symbol/histoday?fsym={}&tsym={}&limit=2000"\
        .format(syms.upper(), comp_syms.upper())
    
    if exchange:
        url += f'&e={exchange}'
        
    url += f"&api_key={api_key}"
    
    response = requests.get(url)
    data = response.json()['Data']
    crypto_df = pd.DataFrame(data)
    # crypto_df['timestamp'] = [datetime.datetime.fromtimestamp(d) for d in crypto_df["timestamp"]]
    
    return crypto_df

In [77]:
df = RetrieveDailyPrice(syms="BTC", comp_syms="USD")

In [78]:
df

,time,top_tier_volume_quote,top_tier_volume_base,top_tier_volume_total,cccagg_volume_quote,cccagg_volume_base,cccagg_volume_total,total_volume_quote,total_volume_base,total_volume_total
0,1502236800,5.800538e+08,6.646535e+08,1.244707e+09,6.578777e+08,8.132512e+08,1.471129e+09,1.315283e+09,1.987069e+09,3.302352e+09
1,1502323200,7.994241e+08,4.498383e+08,1.249262e+09,8.827011e+08,5.775272e+08,1.460228e+09,1.753017e+09,1.368154e+09,3.121172e+09
2,1502409600,9.808912e+08,7.120837e+08,1.692975e+09,1.056087e+09,9.111965e+08,1.967283e+09,2.117083e+09,2.103724e+09,4.220807e+09
3,1502496000,8.789740e+08,1.072526e+09,1.951500e+09,9.745108e+08,1.296806e+09,2.271317e+09,1.936443e+09,3.262563e+09,5.199006e+09
4,1502582400,1.094827e+09,1.613116e+09,2.707943e+09,1.179711e+09,1.994132e+09,3.173843e+09,2.358765e+09,5.036273e+09,7.395038e+09
...,...,...,...,...,...,...,...,...,...,...
1996,1674691200,1.012228e+09,1.852831e+10,1.954054e+10,5.755481e+08,1.371510e+10,1.429065e+10,6.957909e+10,2.015259e+10,8.973168e+10
1997,1674777600,1.003926e+09,1.788451e+10,1.888843e+10,5.895315e+08,1.339189e+10,1.398142e+10,7.206576e+10,1.916666e+10,9.123242e+10
1998,1674864000,9.793524e+08,1.080901e+10,1.178837e+10,5.106967e+08,6.646005e+09,7.156701e+09,7.155615e+10,1.177354e+10,8.332969e+10
1999,1674950400,1.089232e+09,2.030637e+10,2.139560e+10,6.049341e+08,1.453938e+10,1.514431e+10,7.327177e+10,2.197345e+10,9.524521e+10
